In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import socket, struct
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns', 500)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
%pip install pyarrow
%pip install fastparquet

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


Note: you may need to restart the kernel to use updated packages.


In [6]:
df = pd.read_parquet('merged_dataset.gzip')

In [7]:
def ip2long(ip):
    """
    Convert an IP string to long
    """
    packedIP = socket.inet_aton(ip)
    return struct.unpack("!L", packedIP)[0]

# Convert IPs to numbers
df.loc[:,'IPV4_SRC_ADDR'] = df['IPV4_SRC_ADDR'].copy().apply(ip2long)
df.loc[:,'IPV4_DST_ADDR'] = df['IPV4_DST_ADDR'].copy().apply(ip2long)
df.head(2)

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,MIN_IP_PKT_LEN,MAX_IP_PKT_LEN,SRC_TO_DST_SECOND_BYTES,DST_TO_SRC_SECOND_BYTES,RETRANSMITTED_IN_BYTES,RETRANSMITTED_IN_PKTS,RETRANSMITTED_OUT_BYTES,RETRANSMITTED_OUT_PKTS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,Label,Attack,L7_PROTO_NAME,PROTOCOL_MAP
0,3232261268,65389,3232261127,80,6,420,3,0,0,2,4293092,0,140,140280.0,0.0,140,1,0,0,512,0,1,DoS,CBT,tcp
1,3232261268,11154,3232261125,80,6,280,2,40,1,22,4294499,40,140,280.0,40.0,0,0,0,0,512,0,1,DoS,CBT,tcp


In [8]:
# Remove rows with NaN Labels
df = df[df['L7_PROTO_NAME'].notna()]
len(df[df.Label == 0]), len(df[df.Label == 1]), df.Label.isna().sum()

(2421738, 2555250, 0)

In [10]:
def spb_to_mean(x):
    vals = x.split(',')
    vals = list(filter(lambda v: v != '', vals))
    vals = [int(float(x)) for x in vals]
    
    if len(vals) == 0: return 0
    
    return np.asarray(vals, dtype=np.float64).mean()

def spb_to_total(x):
    vals = x.split(',')
    vals = list(filter(lambda v: v != '', vals))
    vals = [int(float(x)) for x in vals]
    
    return np.asarray(vals, dtype=np.float64).sum()

df['DST_TO_SRC_SECOND_BYTES_MEAN'] = df.DST_TO_SRC_SECOND_BYTES.apply(spb_to_mean)
df['DST_TO_SRC_SECOND_BYTES_TOTAL'] = df.DST_TO_SRC_SECOND_BYTES.apply(spb_to_total)
df['SRC_TO_DST_SECOND_BYTES_MEAN'] = df.SRC_TO_DST_SECOND_BYTES.apply(spb_to_mean)
df['SRC_TO_DST_SECOND_BYTES_TOTAL'] = df.SRC_TO_DST_SECOND_BYTES.apply(spb_to_total)